# Getting started

## Quickstart

In [ ]:
from lexy_py import LexyClient

lexy = LexyClient()

We can get more information about the Lexy instance by calling the `info` method. Here we see existing Collections, Indexes, Transformers, and Bindings.

In [ ]:
lexy.info()

There are several documents already in the "**default**" collection. We can list them with the `list_documents` method.

In [ ]:
lexy.list_documents()

The documents in the "**default**" collection are automatically embedded, and the embeddings are stored in the index "**default_text_embeddings**".

In [ ]:
lexy.indexes

We can query the index for "_what is deep learning_" and see our documents ranked by cosine similarity.

In [ ]:
lexy.query_index('what is deep learning')

## Extended example: Famous biographies

Let's go through a longer example to see how we can use Lexy to create and query embeddings for a new collection of documents.

### Collections and Documents

We can see that there are currently two collections, "**default**" and "**code**". Let's create a new collection for famous biographies.

In [ ]:
lexy.collections

In [ ]:
bios = lexy.create_collection('bios', description='Famous biographies')
bios

We can see that our new collection is now listed in the `collections` property, and that it contains no documents.

In [ ]:
lexy.collections

In [ ]:
bios.list_documents()

Let's add a few documents to our new collection.

In [ ]:
bios.add_documents([
    {"content": "Stephen Curry is an American professional basketball player for the Golden State Warriors."},
    {"content": "Dwayne 'The Rock' Johnson is a well-known actor, former professional wrestler, and businessman."},
    {"content": "Taylor Swift is a singer known for her songwriting, musical versatility, and artistic reinventions."}
])

### Indexes and Bindings

Now we want to create embeddings for the documents in our new collection. We'll use the "**text.embeddings.minilm**" transformer, which uses the [MiniLM sentence transformer](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) model to generate embeddings for text.

In [ ]:
lexy.transformers

Before we can bind this transformer to our collection, we need to create an index for storing the resulting embeddings. Let's create a new index called "**bios_index**" with embeddings for our new collection. Our index will have a single field called "**bio_embedding**" that will store the embeddings output by the MiniLM sentence transformer.

In [ ]:
index_fields = {
    "bio_embedding": {"type": "embedding", "extras": {"dims": 384, "distance": "cosine"}}
}
index = lexy.create_index(index_id='bios_index', description='Biography embeddings', index_fields=index_fields)
index

In [ ]:
lexy.indexes

Now let's create a new binding. Our binding will feed the documents in our "**bios**" collection into the "**text.embeddings.minilm**" transformer, and insert the resulting output in our newly created index, "**bios_index**".
<br>
<h5><center>`bios` Collection &nbsp;&nbsp;&rarr;&nbsp;&nbsp; `text.embeddings.minilm` Transformer &nbsp;&nbsp;&rarr;&nbsp;&nbsp; `bios_index` Index </center></h5>
<br>

In [ ]:
binding = lexy.create_binding(collection_id='bios',
                              transformer_id='text.embeddings.minilm',
                              index_id='bios_index')
binding

Our binding automatically runs asynchronous jobs to process our documents and store the results in our index as embeddings. We can now query our index for "_famous artists_" and see the results ranked by cosine similarity.

In [ ]:
index.query('famous artists', query_field='bio_embedding', k=3)

Because our binding has status set to "`ON`", any new documents added to our collection will automatically be processed by our transformer and inserted into our index. We can add another document, and then run the same query to see the new results.

In [ ]:
bios.add_documents([
    {"content": "Beyoncé is a singer and songwriter recognized for her boundary-pushing artistry, vocals, and performances."}
])

In [ ]:
index.query('famous artists', query_field='bio_embedding', k=3)

### Transformers

So far, we've only used the default transformers included in Lexy. Let's see how we can easily create our own transformers.

_Coming soon._